<a href="https://colab.research.google.com/github/AKUMON12/Elvinia_Activity-7-Data-Encryption/blob/master/Elvinia_Lab4Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic data for loan approval
data = {
    'Loan_ID': np.arange(1, 101),
    'Gender': np.random.choice(['Male', 'Female'], 100),
    'Married': np.random.choice(['Yes', 'No'], 100),
    'Dependents': np.random.choice([0, 1, 2, 3], 100),
    'Education': np.random.choice(['Graduate', 'Not Graduate'], 100),
    'ApplicantIncome': np.random.randint(2000, 15000, 100),
    'CoapplicantIncome': np.random.randint(0, 5000, 100),
    'LoanAmount': np.random.randint(100, 700, 100),
    'Loan_Amount_Term': np.random.choice([360, 180, 120], 100),
    'Credit_History': np.random.choice([1, 0], 100),
    'Property_Area': np.random.choice(['Urban', 'Semiurban', 'Rural'], 100),
    'Loan_Status': np.random.choice(['Y', 'N'], 100)
}

# Create a DataFrame
loan_data = pd.DataFrame(data)

# Save the dataset as a CSV file
loan_data.to_csv('loan_approval_data.csv', index=False)

print("Synthetic dataset 'loan_approval_data.csv' created successfully.")

Synthetic dataset 'loan_approval_data.csv' created successfully.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load synthetic dataset (or downloaded one)
url = 'loan_approval_data.csv'
df = pd.read_csv(url)

# Preprocessing: Encode categorical data
df_encoded = pd.get_dummies(df, drop_first=True)

# Features and target
X = df_encoded.drop('Loan_Status_Y', axis=1)
y = df_encoded['Loan_Status_Y']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train decision tree model
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.43333333333333335
Confusion Matrix:
 [[ 7  6]
 [11  6]]


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load synthetic dataset (or downloaded one)
url = 'loan_approval_data.csv'
df = pd.read_csv(url)

# Preprocessing: Encode categorical data
df_encoded = pd.get_dummies(df, drop_first=True)

# Features and target
X = df_encoded.drop('Loan_Status_Y', axis=1)
y = df_encoded['Loan_Status_Y']

# Corrected split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train decision tree model
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.43333333333333335
Confusion Matrix:
 [[ 6  7]
 [10  7]]


In [4]:
# Add predictions to test set for analysis
test_with_predictions = X_test.copy()
test_with_predictions['Actual_Gender'] = df.iloc[y_test.index]['Gender']  # Get original gender
test_with_predictions['Predicted_Loan_Status'] = y_pred

# Calculate approval rates by gender
gender_bias_check = test_with_predictions.groupby('Actual_Gender')['Predicted_Loan_Status'].mean()
print("Approval Rate by Gender:")
print(gender_bias_check)

Approval Rate by Gender:
Actual_Gender
Female    0.526316
Male      0.363636
Name: Predicted_Loan_Status, dtype: float64


In [5]:
from sklearn.utils import resample

# Resample the minority class (Female)
df_minority = df[df['Gender'] == 'Female']
df_majority = df[df['Gender'] == 'Male']

df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=42)
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Preprocess the balanced dataset
df_balanced_encoded = pd.get_dummies(df_balanced, drop_first=True)

# Features and target
X_bal = df_balanced_encoded.drop('Loan_Status_Y', axis=1)
y_bal = df_balanced_encoded['Loan_Status_Y']

# Split and retrain the model
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_bal, y_bal, test_size=0.3, random_state=42)
clf_bal = DecisionTreeClassifier()
clf_bal.fit(X_train_bal, y_train_bal)

# Evaluate balanced model
y_pred_bal = clf_bal.predict(X_test_bal)
print("Balanced Accuracy:", accuracy_score(y_test_bal, y_pred_bal))
print("Confusion Matrix:\n", confusion_matrix(y_test_bal, y_pred_bal))

Balanced Accuracy: 0.48148148148148145
Confusion Matrix:
 [[ 2  8]
 [ 6 11]]


In [8]:
# Check fairness in balanced model
balanced_test_with_predictions = X_test_bal.copy()

# Get the original indices from X_test_bal before resetting
original_indices = X_test_bal.index

# Reset index of X_test_bal for correct alignment
balanced_test_with_predictions = balanced_test_with_predictions.reset_index(drop=True)
y_test_bal_reset = y_test_bal.reset_index(drop=True)

# Use the original indices to get the 'Gender' column from df_balanced
balanced_test_with_predictions['Actual_Gender'] = df_balanced.loc[original_indices, 'Gender'].reset_index(drop=True)
balanced_test_with_predictions['Predicted_Loan_Status'] = y_pred_bal

balanced_fairness = balanced_test_with_predictions.groupby('Actual_Gender')['Predicted_Loan_Status'].mean()
print("Approval Rate by Gender (Balanced Model):")
print(balanced_fairness)

Approval Rate by Gender (Balanced Model):
Actual_Gender
Female    0.736842
Male      0.625000
Name: Predicted_Loan_Status, dtype: float64


In [9]:
# Remove PII (Loan_ID, Name, etc.)
df_no_pii = df.drop(['Loan_ID'], axis=1)

# Preprocess the dataset without PII
df_no_pii_encoded = pd.get_dummies(df_no_pii, drop_first=True)

# Features and target
X_no_pii = df_no_pii_encoded.drop('Loan_Status_Y', axis=1)
y_no_pii = df_no_pii_encoded['Loan_Status_Y']

# Split and retrain the model
X_train_no_pii, X_test_no_pii, y_train_no_pii, y_test_no_pii = train_test_split(X_no_pii, y_no_pii, test_size=0.3, random_state=42)
clf_no_pii = DecisionTreeClassifier()
clf_no_pii.fit(X_train_no_pii, y_train_no_pii)

# Evaluate model without PII
y_pred_no_pii = clf_no_pii.predict(X_test_no_pii)
print("Accuracy without PII:", accuracy_score(y_test_no_pii, y_pred_no_pii))

Accuracy without PII: 0.4


In [10]:
# Compare accuracy with and without PII
original_accuracy = accuracy_score(y_test, y_pred)
no_pii_accuracy = accuracy_score(y_test_no_pii, y_pred_no_pii)

print(f"Original Model Accuracy: {original_accuracy}")
print(f"Model Accuracy without PII: {no_pii_accuracy}")

if abs(original_accuracy - no_pii_accuracy) < 0.05:
    print("Removing PII did not significantly affect model performance.")
else:
    print("Removing PII had a noticeable impact on model performance.")

Original Model Accuracy: 0.43333333333333335
Model Accuracy without PII: 0.4
Removing PII did not significantly affect model performance.
